# Deep Learning Project 1: Computer Vision with PyTorch

## Student Information

- **Name:** Shreya Raghunath
- **Student ID:** 2021771444
- **Date:** 20/07/2025

## Project Overview

This project demonstrates the complete PyTorch workflow for computer vision tasks using the CIFAR-10 dataset. We will build, train, and evaluate multiple neural network architectures to classify images into 10 different categories.

### Learning Objectives
1. Understand the PyTorch workflow for computer vision
2. Implement different neural network architectures
3. Train and evaluate models on real image data
4. Compare model performance and understand trade-offs
5. Save and load trained models

### Dataset: CIFAR-10
The CIFAR-10 dataset consists of 60,000 32x32 color images in 10 different classes:
- Airplane, Automobile, Bird, Cat, Deer
- Dog, Frog, Horse, Ship, Truck

Each class has 6,000 images (5,000 for training, 1,000 for testing).

## Step 1: Setup and Imports

We begin by importing all the necessary libraries for data handling, model building, training, evaluation, and visualization.

In [ ]:
# Import necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import time
import random
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Set random seeds for reproducibility
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)

# Check if CUDA is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")
print(f"torchvision version: {torchvision.__version__}")

## Step 2: Data Preparation

We load the CIFAR-10 dataset using `torchvision.datasets` and apply appropriate transformations:

- **ToTensor**: Converts images to tensors
- **Normalization**: Scales pixel values to a range that accelerates training

**We also define:**
- **Training**, **validation**, and **test** splits
- **Data loaders** for batch processing

**Findings:**
-	ToTensor() scaled images to [0, 1] and converted them to PyTorch tensors.
-	Normalization with CIFAR-10-specific mean and std ([0.4914, 0.4822, 0.4465], [0.247, 0.243, 0.261]) improved model convergence.
-	Proper batching helped in speeding up GPU training.


In [ ]:
# Define data transformations
# We'll normalize the images to help with training stability
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Data augmentation
    transforms.RandomRotation(10),      # Data augmentation
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

# Load CIFAR-10 dataset
print("Loading CIFAR-10 dataset...")
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform_test)

# Create data loaders
batch_size = 128
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

# CIFAR-10 classes
classes = ('airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

print(f"Training samples: {len(trainset)}")
print(f"Test samples: {len(testset)}")
print(f"Number of classes: {len(classes)}")
print(f"Batch size: {batch_size}")

## Step 3: Data Visualization

Before training, we visualize a batch of images and their corresponding class labels to confirm successful loading and to get a feel for the image quality and variability.

**Findings:**
- Dataset loaded correctly.
- Classes appear well-distributed.
- No image corruption or skew observed.



In [ ]:
# Function to show images
def imshow(img):
    # Denormalize the image
    img = img / 2 + 0.5  # Unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.axis('off')

# Get some random training images
dataiter = iter(trainloader)
images, labels = next(dataiter)

# Show images
plt.figure(figsize=(12, 8))
for i in range(16):
    plt.subplot(4, 4, i+1)
    imshow(torchvision.utils.make_grid(images[i]))
    plt.title(classes[labels[i]])
plt.tight_layout()
plt.show()

# Print some statistics
print(f"Image shape: {images[0].shape}")
print(f"Number of channels: {images[0].shape[0]}")
print(f"Image height: {images[0].shape[1]}")
print(f"Image width: {images[0].shape[2]}")

## Step 4: Model 1 - Simple Feedforward Neural Network

As a baseline, we train a **fully connected feedforward neural network** (FCNN) on flattened image vectors.

This approach helps us:
- Establish a benchmark before using CNNs
- Observe the limitations of FCNNs for image data (e.g., lack of spatial locality awareness)

**Findings :**

- FCNN lacks ability to capture spatial patterns.
- Low accuracy (~40-50%) due to flattened image input.


In [ ]:
class SimpleNN(nn.Module):
    def __init__(self, input_size=3072, hidden_size=512, num_classes=10):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Initialize the model
model1 = SimpleNN().to(device)
print("Model 1 - Simple Feedforward Neural Network:")
print(model1)
print(f"Total parameters: {sum(p.numel() for p in model1.parameters()):,}")

## Step 5: Training Function

Let's create a training function that we can reuse for all our models.

In [ ]:
def train_model(model, trainloader, testloader, epochs=10, learning_rate=0.001):
    """
    Train a PyTorch model and return training history
    """
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Lists to store metrics
    train_losses = []
    train_accuracies = []
    test_accuracies = []
    
    print(f"Training for {epochs} epochs...")
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            # Statistics
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Print progress every 100 batches
            if i % 100 == 99:
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 100:.3f}')
                running_loss = 0.0
        
        # Calculate training accuracy
        train_accuracy = 100 * correct / total
        train_losses.append(running_loss / len(trainloader))
        train_accuracies.append(train_accuracy)
        
        # Evaluation phase
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in testloader:
                images, labels = data[0].to(device), data[1].to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        test_accuracy = 100 * correct / total
        test_accuracies.append(test_accuracy)
        
        print(f'Epoch {epoch+1}/{epochs} - Train Acc: {train_accuracy:.2f}%, Test Acc: {test_accuracy:.2f}%')
    
    print('Finished Training!')
    
    return {
        'train_losses': train_losses,
        'train_accuracies': train_accuracies,
        'test_accuracies': test_accuracies
    }

## Step 6: Train Model 1

Now let's train our first model and see how it performs.

**Findings:**

- Training function worked across models.
- Overfitting observed due to model limitations.


In [ ]:
# Train Model 1
print("Training Model 1 - Simple Feedforward Neural Network")
print("=" * 60)

history1 = train_model(model1, trainloader, testloader, epochs=10, learning_rate=0.001)

# Plot training results
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history1['train_accuracies'], label='Training Accuracy')
plt.plot(history1['test_accuracies'], label='Test Accuracy')
plt.title('Model 1 - Accuracy Over Time')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history1['train_losses'], label='Training Loss')
plt.title('Model 1 - Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"Final Test Accuracy: {history1['test_accuracies'][-1]:.2f}%")

## Step 7: Model 2 - Convolutional Neural Network (CNN)

We now implement a CNN that leverages spatial information in images using convolutional and pooling layers. CNNs are better suited for image classification tasks.

**Findings:**

- CNN outperformed FCNN (accuracy ~70-80%).
- Use of Conv2D, MaxPooling2D, Dropout, and Flatten enabled deep feature learning.


In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        
        # Pooling layer
        self.pool = nn.MaxPool2d(2, 2)
        
        # Dropout for regularization
        self.dropout = nn.Dropout(0.25)
        
        # Fully connected layers
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, num_classes)
        
        # Activation function
        self.relu = nn.ReLU()
        
    def forward(self, x):
        # First conv block
        x = self.pool(self.relu(self.conv1(x)))
        
        # Second conv block
        x = self.pool(self.relu(self.conv2(x)))
        
        # Third conv block
        x = self.pool(self.relu(self.conv3(x)))
        
        # Flatten the output
        x = x.view(-1, 128 * 4 * 4)
        
        # Fully connected layers
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        
        return x

# Initialize the CNN model
model2 = CNN().to(device)
print("Model 2 - Convolutional Neural Network:")
print(model2)
print(f"Total parameters: {sum(p.numel() for p in model2.parameters()):,}")

## Step 8: Train Model 2

Let's train our CNN model and compare its performance with the simple feedforward network.

In [ ]:
# Train Model 2
print("Training Model 2 - Convolutional Neural Network")
print("=" * 60)

history2 = train_model(model2, trainloader, testloader, epochs=15, learning_rate=0.001)

# Plot training results
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history2['train_accuracies'], label='Training Accuracy')
plt.plot(history2['test_accuracies'], label='Test Accuracy')
plt.title('Model 2 - CNN Accuracy Over Time')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history2['train_losses'], label='Training Loss')
plt.title('Model 2 - CNN Loss Over Time')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

print(f"Final Test Accuracy: {history2['test_accuracies'][-1]:.2f}%")

## Step 9: Model Comparison

Let's compare the performance of both models to understand the differences.

**Findings:**

- Plotting test accuracy over epochs for Simple NN and CNN → CNN consistently achieves higher test accuracy, ending at 80.73% vs 45.50% for Simple NN.

- Plotting training accuracy over epochs for both models → CNN shows stronger learning with steadily increasing training accuracy.

- CNN clearly outperforms Simple NN in both metrics.

- CNN shows a 35.23 percentage point improvement over Simple NN.


In [ ]:
# Compare model performances
plt.figure(figsize=(15, 5))

# Plot test accuracies
plt.subplot(1, 3, 1)
plt.plot(history1['test_accuracies'], label='Simple NN', marker='o')
plt.plot(history2['test_accuracies'], label='CNN', marker='s')
plt.title('Test Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Test Accuracy (%)')
plt.legend()
plt.grid(True)

# Plot training accuracies
plt.subplot(1, 3, 2)
plt.plot(history1['train_accuracies'], label='Simple NN', marker='o')
plt.plot(history2['train_accuracies'], label='CNN', marker='s')
plt.title('Training Accuracy Comparison')
plt.xlabel('Epoch')
plt.ylabel('Training Accuracy (%)')
plt.legend()
plt.grid(True)

# Bar plot of final accuracies
plt.subplot(1, 3, 3)
models = ['Simple NN', 'CNN']
final_test_acc = [history1['test_accuracies'][-1], history2['test_accuracies'][-1]]
final_train_acc = [history1['train_accuracies'][-1], history2['train_accuracies'][-1]]

x = np.arange(len(models))
width = 0.35

plt.bar(x - width/2, final_train_acc, width, label='Training Accuracy', alpha=0.8)
plt.bar(x + width/2, final_test_acc, width, label='Test Accuracy', alpha=0.8)

plt.xlabel('Models')
plt.ylabel('Accuracy (%)')
plt.title('Final Model Performance')
plt.xticks(x, models)
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary statistics
print("Model Performance Summary:")
print("=" * 50)
print(f"Simple NN - Final Test Accuracy: {history1['test_accuracies'][-1]:.2f}%")
print(f"CNN - Final Test Accuracy: {history2['test_accuracies'][-1]:.2f}%")
print(f"Improvement: {history2['test_accuracies'][-1] - history1['test_accuracies'][-1]:.2f} percentage points")

##  Step 10: Confusion Matrix

The confusion matrix gives a visual breakdown of correct vs incorrect predictions per class. It helps identify which classes the model confuses most often.

**Findings:**

- Evaluates the trained CNN model on the test dataset using evaluate_model() → returns predictions, true labels, and overall accuracy.

- Computes and displays the confusion matrix → shows correct vs incorrect predictions for each class. Most classes like automobile, frog, ship, truck have strong diagonals (high correct counts), indicating good performance.

- Calculates per-class accuracy from confusion matrix → cat has the lowest at 58.2%, while ship has the highest at 90.5%, revealing class-wise variation in performance.

- Visualizes per-class accuracy as a bar chart → quickly highlights which classes the model performs best and worst on.

- overall accuracy is 81%, with solid metrics for most classes, though cat and bird show relatively lower f1-scores.


In [ ]:
def evaluate_model(model, testloader, classes):
    """
    Evaluate model and return predictions, true labels, and confusion matrix
    """
    model.eval()
    all_predictions = []
    all_labels = []
    correct = 0
    total = 0
    
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_predictions.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    accuracy = 100 * correct / total
    
    return all_predictions, all_labels, accuracy

# Evaluate the CNN model
print("Evaluating CNN Model...")
predictions, true_labels, accuracy = evaluate_model(model2, testloader, classes)

print(f"Overall Accuracy: {accuracy:.2f}%")

# Create confusion matrix
cm = confusion_matrix(true_labels, predictions)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=classes, yticklabels=classes)
plt.title('Confusion Matrix - CNN Model')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Calculate per-class accuracy
class_accuracy = cm.diagonal() / cm.sum(axis=1) * 100

plt.figure(figsize=(12, 6))
bars = plt.bar(classes, class_accuracy, color='skyblue', alpha=0.7)
plt.title('Per-Class Accuracy - CNN Model')
plt.xlabel('Classes')
plt.ylabel('Accuracy (%)')
plt.xticks(rotation=45)
plt.ylim(0, 100)

# Add value labels on bars
for bar, acc in zip(bars, class_accuracy):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
             f'{acc:.1f}%', ha='center', va='bottom')

plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Print classification report
print("\nClassification Report:")
print(classification_report(true_labels, predictions, target_names=classes))

## Step 11: Model Predictions on Sample Images

Let's test our model on some sample images to see how it performs visually.

**Findings:**

- Selects a few test images from the dataset and passes them through the trained CNN model → generates predictions for visual inspection.

- Denormalizes images before displaying → triggers a warning because some pixel values fall outside the valid display range, but plots still render correctly.

- Displays the sample images in a 2x4 grid with predicted vs true labels → uses green titles for correct predictions (✓) and red for incorrect ones (✗).

- Helps visually assess model performance on real samples → shows that the model makes mostly correct predictions, with occasional misclassifications.

In [ ]:
def predict_and_visualize(model, testloader, num_samples=8):
    """
    Make predictions on sample images and visualize results
    """
    model.eval()
    
    # Get a batch of test images
    dataiter = iter(testloader)
    images, labels = next(dataiter)
    
    # Make predictions
    with torch.no_grad():
        outputs = model(images[:num_samples].to(device))
        _, predicted = torch.max(outputs, 1)
    
    # Plot results
    plt.figure(figsize=(15, 10))
    for i in range(num_samples):
        plt.subplot(2, 4, i+1)
        
        # Denormalize image
        img = images[i] / 2 + 0.5
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        
        # Set title with prediction result
        true_label = classes[labels[i]]
        pred_label = classes[predicted[i]]
        
        if true_label == pred_label:
            color = 'green'
            title = f'✓ {pred_label}\n(True: {true_label})'
        else:
            color = 'red'
            title = f'✗ {pred_label}\n(True: {true_label})'
        
        plt.title(title, color=color, fontsize=10)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Test predictions on sample images
print("Testing CNN Model on Sample Images:")
predict_and_visualize(model2, testloader, num_samples=8)

## Step 12: Save and Load Model

Let's save our best performing model and demonstrate how to load it back.

In [ ]:
# Save the best model
print("Saving the CNN model...")
torch.save({
    'model_state_dict': model2.state_dict(),
    'optimizer_state_dict': optim.Adam(model2.parameters()).state_dict(),
    'epoch': 15,
    'accuracy': history2['test_accuracies'][-1],
    'model_class': CNN
}, 'cifar10_cnn_model.pth')

print("Model saved successfully!")

# Load the model back
print("\nLoading the saved model...")
checkpoint = torch.load('cifar10_cnn_model.pth', weights_only=False)  # Add weights_only=False

# Create a new model instance
loaded_model = CNN().to(device)
loaded_model.load_state_dict(checkpoint['model_state_dict'])

# Test the loaded model
loaded_model.eval()
with torch.no_grad():
    # Get a single batch
    dataiter = iter(testloader)
    images, labels = next(dataiter)
    
    # Make prediction
    outputs = loaded_model(images[:1].to(device))
    _, predicted = torch.max(outputs, 1)
    
    print(f"Sample prediction - Predicted: {classes[predicted[0]]}, True: {classes[labels[0]]}")
    print(f"Model accuracy from checkpoint: {checkpoint['accuracy']:.2f}%")

print("Model loaded and tested successfully!")

## Step 13: Model Analysis and Insights

Let's analyze our models and draw some insights from the results.

**Findings:**

- Compares total trainable parameters → CNN has fewer parameters (1.15M) than Simple NN (1.57M), meaning it's more efficient at learning with fewer weights.

- Evaluates model performance → CNN outperforms Simple NN with a 35.23 percentage point higher final accuracy (80.73% vs 45.50%).

- Looks at training duration → CNN trains longer (15 epochs) but achieves much better accuracy, showing effective learning over time.

- Analyzes overfitting by comparing train and test accuracy → Simple NN slightly underfits (-2.32%), while CNN slightly overfits (4.63%), yet generalizes well overall.

In [ ]:
# Model complexity comparison
model1_params = sum(p.numel() for p in model1.parameters())
model2_params = sum(p.numel() for p in model2.parameters())

print("Model Complexity Analysis:")
print("=" * 40)
print(f"Simple NN Parameters: {model1_params:,}")
print(f"CNN Parameters: {model2_params:,}")
print(f"Parameter Ratio (CNN/Simple): {model2_params/model1_params:.2f}x")

# Performance comparison
print(f"\nPerformance Comparison:")
print("=" * 40)
print(f"Simple NN Final Accuracy: {history1['test_accuracies'][-1]:.2f}%")
print(f"CNN Final Accuracy: {history2['test_accuracies'][-1]:.2f}%")
print(f"Accuracy Improvement: {history2['test_accuracies'][-1] - history1['test_accuracies'][-1]:.2f} percentage points")

# Training time analysis (approximate)
print(f"\nTraining Efficiency:")
print("=" * 40)
print(f"Simple NN Training Epochs: 10")
print(f"CNN Training Epochs: 15")
print(f"CNN achieved higher accuracy in more epochs")

# Overfitting analysis
simple_nn_overfit = history1['train_accuracies'][-1] - history1['test_accuracies'][-1]
cnn_overfit = history2['train_accuracies'][-1] - history2['test_accuracies'][-1]

print(f"\nOverfitting Analysis:")
print("=" * 40)
print(f"Simple NN Overfitting: {simple_nn_overfit:.2f} percentage points")
print(f"CNN Overfitting: {cnn_overfit:.2f} percentage points")

if cnn_overfit < simple_nn_overfit:
    print("CNN shows better generalization (less overfitting)")
else:
    print("Simple NN shows better generalization (less overfitting)")

## Step 14: Conclusion and Summary

In this project, we built and improved deep learning models for CIFAR-10 image classification. We observed how CNNs outperform FCNNs and learned how regularization and data augmentation enhance generalization. This hands-on experience strengthens our understanding of model development, tuning, and evaluation in computer vision.


In [ ]:
# Final summary
print("Deep Learning Project 1 - Summary")
print("=" * 50)
print("\nWhat we accomplished:")
print("1. ✅ Set up PyTorch environment and loaded CIFAR-10 dataset")
print("2. ✅ Built and trained a simple feedforward neural network")
print("3. ✅ Built and trained a convolutional neural network (CNN)")
print("4. ✅ Compared model performances and analyzed results")
print("5. ✅ Evaluated the best model with confusion matrix and per-class accuracy")
print("6. ✅ Saved and loaded the trained model")
print("7. ✅ Analyzed model complexity and overfitting")

print("\nKey Findings:")
print(f"• CNN achieved {history2['test_accuracies'][-1]:.2f}% accuracy vs {history1['test_accuracies'][-1]:.2f}% for simple NN")
print(f"• CNN has {model2_params:,} parameters vs {model1_params:,} for simple NN")
print(f"• CNN shows {'better' if cnn_overfit < simple_nn_overfit else 'worse'} generalization")

print("\nLessons Learned:")
print("• Convolutional layers are essential for image classification tasks")
print("• Data augmentation helps improve model generalization")
print("• Proper model evaluation requires multiple metrics")
print("• Model complexity should be balanced with performance")


## References and Resources

1. **PyTorch Documentation**: https://pytorch.org/docs/
2. **CIFAR-10 Dataset**: https://www.cs.toronto.edu/~kriz/cifar.html
3. **Convolutional Neural Networks**: https://cs231n.github.io/convolutional-networks/
4. **Deep Learning Fundamentals**: https://www.deeplearningbook.org/

---